In [2]:
%load_ext autoreload
%autoreload 2
import torch

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import save_target_question_ids
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np

from jaxtyping import Float
from torch import Tensor

import plotly.express as px


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# resid pre 9
REPO_ID = "eoinf/unlearning_saes"
FILENAME = "jolly-dream-40/sparse_autoencoder_gemma-2b-it_blocks.9.hook_resid_pre_s16384_127995904.pt"


filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)

model = model_store_from_sae(sae)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


#### Add new dataset

In [6]:
for dataset in ['high_school_geography', 'human_aging', 'college_biology']: # ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']
    save_target_question_ids(model, dataset)

100%|██████████| 792/792 [01:53<00:00,  7.00it/s]


Found correct questions:  30
Found correct questions but not correct without question prompt:  29


100%|██████████| 892/892 [02:04<00:00,  7.19it/s]


Found correct questions:  32
Found correct questions but not correct without question prompt:  31


100%|██████████| 576/576 [01:34<00:00,  6.12it/s]

Found correct questions:  15
Found correct questions but not correct without question prompt:  14


: 

#### Usage

In [4]:

filename = "../data/wmdp-bio_gemma_2b_it_correct.csv"
correct_question_ids = np.genfromtxt(filename)

dataset_args = {
    'question_subset': correct_question_ids,
}

sae.cfg.n_batches_in_store_buffer = 86
unlearning_metric = 'wmdp-bio_gemma_2b_it_correct'

# by default, the control metrics are ['high_school_us_history', 'college_computer_science',
# 'high_school_geography', 'human_aging', 'college_biology']
unlearn_cfg = UnlearningConfig(unlearn_activation_store=None, unlearning_metric=unlearning_metric)

# or you can pass in custom control metrics
# # control_metric = ['high_school_us_history', 'college_biology']
# # unlearn_cfg = UnlearningConfig(unlearn_activation_store=None, unlearning_metric=unlearning_metric, control_metric=control_metric)

ul_tool = SAEUnlearningTool(unlearn_cfg)
ul_tool.setup(create_base_act_store=False, create_unlearn_act_store=False, model=model)

In [6]:
features_to_ablate = [7983, 16186, 12273, 14315, 4342, 10051, 15858, 6958]
multiplier = 20

ablate_params = {
    'features_to_ablate': features_to_ablate,
    'multiplier': multiplier,
    'intervention_method': 'scale_feature_activation',
    'permutations': [[0,1,2,3]]
}

# metrics = ul_tool.calculate_metrics(**ablate_params)
control_metrics = ul_tool.calculate_control_metrics(**ablate_params)
# loss_added = ul_tool.compute_loss_added(n_batch=30, **ablate_params)

100%|██████████| 3/3 [00:01<00:00,  2.83it/s]


In [7]:
control_metrics

{'high_school_us_history': {'mean_correct': 0.48148149251937866,
  'total_correct': 13,
  'is_correct': array([0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
         0., 0., 0., 0., 1., 0., 1., 0., 1., 1.], dtype=float32),
  'output_probs': array([[9.8362988e-01, 1.9729647e-04, 2.0095882e-04, 1.5598402e-02],
         [5.2172208e-04, 1.5751764e-04, 9.9605465e-01, 7.5944222e-06],
         [1.7719527e-01, 4.3885824e-03, 7.9074991e-01, 3.6680204e-04],
         [9.9194568e-01, 5.3297589e-07, 2.0780908e-07, 4.3941171e-08],
         [4.1224668e-03, 1.1418777e-04, 9.5532018e-01, 4.6363533e-05],
         [4.5608860e-01, 1.7948341e-02, 4.2773053e-01, 6.0959820e-02],
         [9.9215615e-01, 8.9372548e-07, 1.4806488e-06, 1.6940235e-03],
         [9.1611344e-01, 7.5462711e-05, 8.0279678e-02, 6.9963098e-06],
         [7.2069401e-05, 2.9160106e-03, 9.9315602e-01, 8.4834446e-06],
         [2.6987885e-07, 9.9895644e-01, 8.2351198e-06, 1.6978831e-07],
         [1.0930685e-04, 1.00